In [1]:
# from parallel_uncertainty_framework import uncertainty_framework
!pwd

/external1/nguyenpham/code/Develop_uncertainty/otdd_explore


In [2]:
home = "/external1/nguyenpham/code/"
import ot
import otdd
import numpy as np
from otdd.pytorch.distance import DatasetDistance
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import pandas as pd
import argparse
import sys
#
torch.cuda.set_device(1)
import math

sys.path.append(f"{home}/Develop_uncertainty/our_method")
from my_gpytorch import settings
from my_gpytorch import utils
from my_gpytorch.kernels import *
from my_gpytorch.mymodels import *
from uncertainty_framework import uncertainty_framework

/home/nguyenpk/miniconda3/envs/resnet_env/lib/python3.9/site-packages/otdd/pytorch/distance.py:23: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
ot.gpu not found - coupling computation will be in cpu


In [3]:
### check kernel distance
import sklearn
import numpy as np
from sklearn.datasets import make_moons , make_circles,make_blobs
from sklearn.metrics import pairwise
from sklearn.metrics import classification_report, confusion_matrix, auc, accuracy_score
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.cluster import kmeans_plusplus
#
from sklearn.metrics import classification_report, confusion_matrix, auc, accuracy_score
from sklearn.preprocessing import LabelEncoder
# from xgboost import XGBClassifier
import numpy as np
import matplotlib.font_manager as font_manager


### CIFAR DATASET


In [7]:
def split_dataset_non_iid(dataset, num_splits, num_classes_per_split=2):
    """
    Splits the dataset into non-IID subsets where each subset contains specific classes.

    Args:
        dataset (torch.utils.data.Dataset): The dataset to split.
        num_splits (int): Number of splits/parties.
        num_classes_per_split (int): Number of classes per split.

    Returns:
        List[Subset]: A list containing the split subsets.
    """
    targets = np.array(dataset.targets)
    classes = np.unique(targets)
    np.random.shuffle(classes)

    classes_per_split = np.array_split(classes, num_splits)
    splits_indices = []

    for split_classes in classes_per_split:
        indices = np.where(np.isin(targets, split_classes))[0]
        np.random.shuffle(indices)
        splits_indices.append(indices)

    splits = [Subset(dataset, indices) for indices in splits_indices]
    return splits

# Example usage for non-IID split
# train_splits_non_iid = split_dataset_non_iid(train_dataset, NUM_PARTIES, num_classes_per_split=2)
# train_loaders_non_iid = [DataLoader(split, batch_size=BATCH_SIZE, shuffle=True) for split in train_splits_non_iid]

In [11]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
import os

# Parameters
NUM_PARTIES = 8
BATCH_SIZE = 64
# DATA_DIR = './data'
SEED = 42

# Set random seed for reproducibility
torch.manual_seed(SEED)
np.random.seed(SEED)

# Define transformations for the dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),  # CIFAR-10 mean
                         (0.2023, 0.1994, 0.2010))  # CIFAR-10 std
])

# Load CIFAR-10 training dataset
train_dataset = datasets.CIFAR10(root=DATA_DIR, train=True, download=True, transform=transform)

# Optionally, load the test dataset
test_dataset = datasets.CIFAR10(root=DATA_DIR, train=False, download=True, transform=transform)

def create_idx_party_mapping(splits, total_size):
    """
    Creates a mapping from each data sample to its party index.

    Args:
        splits (List[List[int]]): List of index lists for each party.
        total_size (int): Total number of samples in the dataset.

    Returns:
        np.ndarray: An array where the ith element is the party index for sample i.
    """
    idx_party = np.zeros(total_size, dtype=int)
    for party_idx, party_indices in enumerate(splits):
        idx_party[party_indices] = party_idx
    return idx_party

def split_dataset_by_indices(dataset, num_parties):
    """
    Splits a dataset into `num_parties` subsets by returning the indices for each party.

    Args:
        dataset (torch.utils.data.Dataset): The dataset to split.
        num_parties (int): Number of parties to split the dataset into.

    Returns:
        List[List[int]]: A list where each element is a list of indices for a party.
    """
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    np.random.shuffle(indices)

    split_sizes = [dataset_size // num_parties] * num_parties
    for i in range(dataset_size % num_parties):
        split_sizes[i] += 1

    splits = []
    start = 0
    for size in split_sizes:
        end = start + size
        splits.append(indices[start:end])
        start = end

    return create_idx_party_mapping(splits, dataset_size)

def split_dataset_non_iid(dataset, num_parties, num_classes_per_party=2):
    """
    Splits the dataset into non-IID subsets where each subset contains specific classes.

    Args:
        dataset (torch.utils.data.Dataset): The dataset to split.
        num_parties (int): Number of parties to split the dataset into.
        num_classes_per_party (int): Number of unique classes per party.

    Returns:
        List[List[int]]: A list where each element is a list of indices for a party.
    """
    dataset_size = len(dataset)
    targets = np.array(dataset.targets)
    classes = np.unique(targets)
    np.random.shuffle(classes)

    # Ensure that the number of classes per party does not exceed total classes
    assert num_parties * num_classes_per_party <= len(classes), "Not enough classes for the specified split."

    splits = []
    for i in range(num_parties):
        selected_classes = classes[i*num_classes_per_party:(i+1)*num_classes_per_party]
        indices = np.where(np.isin(targets, selected_classes))[0]
        np.random.shuffle(indices)
        splits.append(indices.tolist())

    return create_idx_party_mapping(splits, dataset_size)

# Split the training dataset
# idx_ = split_dataset_by_indices(train_dataset, NUM_PARTIES)



Files already downloaded and verified
Files already downloaded and verified


In [29]:
idx1 = split_dataset_non_iid(train_dataset,
                             NUM_PARTIES, 
                             num_classes_per_party=1)

In [23]:
ds = {}

In [24]:
ds['X_train'] = train_dataset.data
ds['y_train'] = train_dataset.targets
ds["X_val"] = test_dataset.data
ds["y_val"] = test_dataset.targets
ds["v_value"] = None
ds["idx_party"] = idx_
pd.to_pickle(ds,"/external1/nguyenpham/code/uncertainty_shapley/data/cifar_iid_8.pickle") 

In [36]:
ds['X_train'] = train_dataset.data
ds['y_train'] = train_dataset.targets
ds["X_val"] = test_dataset.data
ds["y_val"] = test_dataset.targets
ds["v_value"] = None
ds["idx_party"] = idx1
pd.to_pickle(ds,"/external1/nguyenpham/code/uncertainty_shapley/data/cifar_class_8.pickle") 

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import numpy as np
import os

# Parameters
NUM_PARTIES = 8
BATCH_SIZE = 64
DATA_DIR = './data'
SEED = 42

# Set random seed for reproducibility
torch.manual_seed(SEED)
np.random.seed(SEED)

# Define a custom PyTorch Dataset for California Housing
class CaliforniaHousingDataset(Dataset):
    def __init__(self, features, targets, transform=None):
        """
        Args:
            features (np.ndarray): Feature matrix.
            targets (np.ndarray): Target vector.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = features
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        sample = {
            'feature': torch.tensor(self.features[idx], dtype=torch.float32),
            'target': torch.tensor(self.targets[idx], dtype=torch.float32)
        }

        if self.transform:
            sample = self.transform(sample)

        return sample

def load_and_prepare_data():
    """
    Loads the California Housing dataset and preprocesses it.

    Returns:
        train_dataset (CaliforniaHousingDataset): Training dataset.
        test_dataset (CaliforniaHousingDataset): Test dataset.
    """
    # Fetch the California Housing dataset
    housing = fetch_california_housing()
    X, y = housing.data, housing.target

    # Split into training and testing sets (e.g., 80-20 split)
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=SEED
    )

    # Feature scaling (Standardization)
    scaler = StandardScaler()
    target_scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    y_train_scaled = target_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
    y_test_scaled = target_scaler.transform(y_test.reshape(-1, 1)).flatten()


    # Create custom datasets
    train_dataset = CaliforniaHousingDataset(X_train, y_train_scaled)
    test_dataset = CaliforniaHousingDataset(X_test, y_test_scaled)

    return train_dataset, test_dataset

In [112]:
train_dataset, test_dataset = load_and_prepare_data()
idx_ = split_dataset_by_indices(train_dataset, NUM_PARTIES)

ds['X_train'] = train_dataset.data
ds['y_train'] = train_dataset.targets
ds["X_val"] = test_dataset.data
ds["y_val"] = test_dataset.targets
ds["v_value"] = None
ds["idx_party"] = idx_
pd.to_pickle(ds,"/external1/nguyenpham/code/uncertainty_shapley/data/calihouse_iid_8.pickle") 

In [113]:
NUM_PARTIES = 3
train_dataset, test_dataset = load_and_prepare_data()
idx_ = split_dataset_by_indices(train_dataset, NUM_PARTIES)

ds['X_train'] = train_dataset.data
ds['y_train'] = train_dataset.targets
ds["X_val"] = test_dataset.data
ds["y_val"] = test_dataset.targets
ds["v_value"] = None
ds["idx_party"] = idx_
pd.to_pickle(ds,"/external1/nguyenpham/code/uncertainty_shapley/data/calihouse_iid_3.pickle") 

## MNIST

In [24]:
DATA_DIR="/external1/nguyenpham/code/uncertainty_shapley/data"

In [25]:
NUM_PARTIES = 6
transform1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),  # MNIST mean and std
])
train_dataset = datasets.MNIST(root=DATA_DIR, train=True, download=True, transform=transform1)

# Optionally, load the test dataset
test_dataset = datasets.MNIST(root=DATA_DIR, train=False, download=True, transform=transform1)

idx_ = split_dataset_by_indices(train_dataset, NUM_PARTIES)
ds = {}
ds['X_train'] = train_dataset.data.double().view(len(train_dataset.targets), -1)
ds['y_train'] = train_dataset.targets
ds["X_val"] = test_dataset.data.double().view(len(test_dataset.targets), -1)
ds["y_val"] = test_dataset.targets
ds["v_value"] = None
ds["idx_party"] = idx_
pd.to_pickle(ds,f"{DATA_DIR}/mnist_iid_{NUM_PARTIES}.pickle") 

Using downloaded and verified file: /external1/nguyenpham/code/uncertainty_shapley/data/MNIST/raw/train-images-idx3-ubyte.gz
Extracting /external1/nguyenpham/code/uncertainty_shapley/data/MNIST/raw/train-images-idx3-ubyte.gz to /external1/nguyenpham/code/uncertainty_shapley/data/MNIST/raw

Using downloaded and verified file: /external1/nguyenpham/code/uncertainty_shapley/data/MNIST/raw/train-labels-idx1-ubyte.gz
Extracting /external1/nguyenpham/code/uncertainty_shapley/data/MNIST/raw/train-labels-idx1-ubyte.gz to /external1/nguyenpham/code/uncertainty_shapley/data/MNIST/raw

Using downloaded and verified file: /external1/nguyenpham/code/uncertainty_shapley/data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting /external1/nguyenpham/code/uncertainty_shapley/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /external1/nguyenpham/code/uncertainty_shapley/data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 1887125.19it/s]


Extracting /external1/nguyenpham/code/uncertainty_shapley/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /external1/nguyenpham/code/uncertainty_shapley/data/MNIST/raw



In [26]:
idx1 = split_dataset_non_iid(train_dataset,
                             NUM_PARTIES, 
                             num_classes_per_party=1)

In [31]:
NUM_PARTIES = 6
idx1 = split_dataset_non_iid(train_dataset,
                             NUM_PARTIES, 
                             num_classes_per_party=1)
ds = {}
ds['X_train'] = train_dataset.data.double().view(len(train_dataset.targets), -1)
ds['y_train'] = train_dataset.targets
ds["X_val"] = test_dataset.data.double().view(len(test_dataset.targets), -1)
ds["y_val"] = test_dataset.targets
ds["v_value"] = None
ds["idx_party"] = idx1
pd.to_pickle(ds,f"{DATA_DIR}/mnist_non_iid_{NUM_PARTIES}.pickle") 